In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import swifter
import random
from datetime import datetime
import datetime
from sklearn import preprocessing

In [2]:
def ep_to_day(ep):
    return datetime.datetime.fromtimestamp(ep).weekday()

In [3]:
all_features = ["text_ tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "enaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "enaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager", "reply_timestamp", "retweet_timestamp",\
                "retweet_with_comment_timestamp", "like_timestamp"]

training = pd.read_csv('s3://recsys-challenge-2020/training.tsv', encoding="utf-8", names = all_features, sep="\x01", nrows = 10000)

In [4]:
training['day'] = training.swifter.apply(lambda x: ep_to_day(x['tweet_timestamp']), axis = 1)

In [5]:
training['reply_bool'] = training.reply_timestamp.fillna(0)
training.loc[training.reply_bool != 0.0, 'reply_bool'] = 1.0

In [6]:
[(c, training.columns.get_loc(c)) for c in training.columns]

[('text_ tokens', 0),
 ('hashtags', 1),
 ('tweet_id', 2),
 ('present_media', 3),
 ('present_links', 4),
 ('present_domains', 5),
 ('tweet_type', 6),
 ('language', 7),
 ('tweet_timestamp', 8),
 ('enaged_with_user_id', 9),
 ('engaged_with_user_follower_count', 10),
 ('engaged_with_user_following_count', 11),
 ('engaged_with_user_is_verified', 12),
 ('engaged_with_user_account_creation', 13),
 ('enaging_user_id', 14),
 ('enaging_user_follower_count', 15),
 ('enaging_user_following_count', 16),
 ('enaging_user_is_verified', 17),
 ('enaging_user_account_creation', 18),
 ('engagee_follows_engager', 19),
 ('reply_timestamp', 20),
 ('retweet_timestamp', 21),
 ('retweet_with_comment_timestamp', 22),
 ('like_timestamp', 23),
 ('day', 24),
 ('reply_bool', 25)]

tweet_type, language, engagee_follows_engager_True, day, engaged_user_id, engagee_user_id

In [7]:
training_subset = training.iloc[:, [6,7,9,10,11,14,15,16,19,24,25]]

In [8]:
training_subset.head()

,tweet_type,language,enaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,enaging_user_id,enaging_user_follower_count,enaging_user_following_count,engagee_follows_engager,day,reply_bool
0,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,3517,590,0000006C3074607050F1339DDCB890BB,27428,600,False,6,0.0
1,TopLevel,B9175601E87101A984A50F8A62A1C374,1F30A85E612A24B426013978ADB7AE46,39786,32978,000005C520010F8917EEAB6F5B6EC1C4,17,77,True,3,0.0
2,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,141726,3232,000006829BEADA9EEA695CF0C334B426,2,29,False,6,0.0
3,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,BEB3C337B266E2C4769065AD64724329,3044,99,000006829BEADA9EEA695CF0C334B426,3,30,False,6,0.0
4,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,141370,3235,000006829BEADA9EEA695CF0C334B426,3,30,False,3,0.0


In [9]:
training_subset.dtypes

tweet_type                            object
language                              object
enaged_with_user_id                   object
engaged_with_user_follower_count       int64
engaged_with_user_following_count      int64
enaging_user_id                       object
enaging_user_follower_count            int64
enaging_user_following_count           int64
engagee_follows_engager                 bool
day                                    int64
reply_bool                           float64
dtype: object

In [10]:
training_subset['engagee_follows_engager'] = training_subset['engagee_follows_engager'].apply(str)
training_subset['day'] = training_subset['day'].apply(str)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
[(c, training_subset.columns.get_loc(c)) for c in training_subset.columns]

[('tweet_type', 0),
 ('language', 1),
 ('enaged_with_user_id', 2),
 ('engaged_with_user_follower_count', 3),
 ('engaged_with_user_following_count', 4),
 ('enaging_user_id', 5),
 ('enaging_user_follower_count', 6),
 ('enaging_user_following_count', 7),
 ('engagee_follows_engager', 8),
 ('day', 9),
 ('reply_bool', 10)]

In [12]:
training_subset_numerical = training_subset.iloc[:, [3,4,6,7]]

In [13]:
x = training_subset_numerical.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

In [14]:
training_data_all = pd.concat([training_subset, df], axis = 1)

In [15]:
[(c, training_data_all.columns.get_loc(c)) for c in training_data_all.columns]

[('tweet_type', 0),
 ('language', 1),
 ('enaged_with_user_id', 2),
 ('engaged_with_user_follower_count', 3),
 ('engaged_with_user_following_count', 4),
 ('enaging_user_id', 5),
 ('enaging_user_follower_count', 6),
 ('enaging_user_following_count', 7),
 ('engagee_follows_engager', 8),
 ('day', 9),
 ('reply_bool', 10),
 (0, 11),
 (1, 12),
 (2, 13),
 (3, 14)]

In [16]:
training_data_all.head()

,tweet_type,language,enaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,enaging_user_id,enaging_user_follower_count,enaging_user_following_count,engagee_follows_engager,day,reply_bool,0,1,2,3
0,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,3517,590,0000006C3074607050F1339DDCB890BB,27428,600,False,6,0.0,0.000031,0.000313,0.257540,0.006159
1,TopLevel,B9175601E87101A984A50F8A62A1C374,1F30A85E612A24B426013978ADB7AE46,39786,32978,000005C520010F8917EEAB6F5B6EC1C4,17,77,True,3,0.0,0.000353,0.017507,0.000160,0.000781
2,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,141726,3232,000006829BEADA9EEA695CF0C334B426,2,29,False,6,0.0,0.001257,0.001716,0.000019,0.000288
3,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,BEB3C337B266E2C4769065AD64724329,3044,99,000006829BEADA9EEA695CF0C334B426,3,30,False,6,0.0,0.000027,0.000053,0.000028,0.000298
4,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,141370,3235,000006829BEADA9EEA695CF0C334B426,3,30,False,3,0.0,0.001253,0.001717,0.000028,0.000298


In [17]:
training_data = training_data_all.iloc[:, [0,1,2,5,8,9,10,11,12,13,14]]

In [18]:
training_data.head()

,tweet_type,language,enaged_with_user_id,enaging_user_id,engagee_follows_engager,day,reply_bool,0,1,2,3
0,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,0000006C3074607050F1339DDCB890BB,False,6,0.0,0.000031,0.000313,0.257540,0.006159
1,TopLevel,B9175601E87101A984A50F8A62A1C374,1F30A85E612A24B426013978ADB7AE46,000005C520010F8917EEAB6F5B6EC1C4,True,3,0.0,0.000353,0.017507,0.000160,0.000781
2,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,000006829BEADA9EEA695CF0C334B426,False,6,0.0,0.001257,0.001716,0.000019,0.000288
3,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,BEB3C337B266E2C4769065AD64724329,000006829BEADA9EEA695CF0C334B426,False,6,0.0,0.000027,0.000053,0.000028,0.000298
4,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,000006829BEADA9EEA695CF0C334B426,False,3,0.0,0.001253,0.001717,0.000028,0.000298


In [19]:
training_data.rename(columns = {0:'engaged_with_user_follower_count',1:'engaged_with_user_following_count',2:'enaging_user_follower_count', 3:'enaging_user_following_count'},inplace = True)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [20]:
X = training_data.drop(['reply_bool'], axis = 1)
y = training_data[['reply_bool']]

In [21]:
X

,tweet_type,language,enaged_with_user_id,enaging_user_id,engagee_follows_engager,day,engaged_with_user_follower_count,engaged_with_user_following_count,enaging_user_follower_count,enaging_user_following_count
0,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,0000006C3074607050F1339DDCB890BB,False,6,3.112946e-05,0.000313,0.257540,0.006159
1,TopLevel,B9175601E87101A984A50F8A62A1C374,1F30A85E612A24B426013978ADB7AE46,000005C520010F8917EEAB6F5B6EC1C4,True,3,3.527001e-04,0.017507,0.000160,0.000781
2,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,000006829BEADA9EEA695CF0C334B426,False,6,1.256527e-03,0.001716,0.000019,0.000288
3,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,BEB3C337B266E2C4769065AD64724329,000006829BEADA9EEA695CF0C334B426,False,6,2.693572e-05,0.000053,0.000028,0.000298
4,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,000006829BEADA9EEA695CF0C334B426,False,3,1.253371e-03,0.001717,0.000028,0.000298
...,...,...,...,...,...,...,...,...,...,...
9995,Retweet,B9175601E87101A984A50F8A62A1C374,4E76346460CF5B0BDAFE15F4F15BD8AB,00231EBA16A507B3DB239D9283C5CFE2,True,4,3.005664e-06,0.000092,0.004300,0.002406
9996,Quote,22C448FF81263D4BAF2A176145EE9EAD,952B0F59D985675106524A8F93A22F1B,0023208D5F6A8BFBEDD60EA389E34FFD,False,0,7.272467e-04,0.017154,0.000263,0.000679
9997,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,187AC59639DA9A6F32F7CD118EDD58F7,0023208D5F6A8BFBEDD60EA389E34FFD,False,6,4.222143e-03,0.000782,0.000263,0.000679
9998,TopLevel,125C57F4FA6D4E110983FB11B52EFD4E,1BC45E70CDD9F4DD7D98A63229189A4E,002320ED0AA42407A190FE2C3F7E336A,True,1,8.954929e-07,0.000059,0.001023,0.001851


In [22]:
y

,reply_bool
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
9995,0.0
9996,0.0
9997,0.0
9998,0.0


In [23]:
X.dtypes

tweet_type                            object
language                              object
enaged_with_user_id                   object
enaging_user_id                       object
engagee_follows_engager               object
day                                   object
engaged_with_user_follower_count     float64
engaged_with_user_following_count    float64
enaging_user_follower_count          float64
enaging_user_following_count         float64
dtype: object

In [24]:
X_numerical = pd.get_dummies(X, sparse = True)

In [25]:
X_numerical

,engaged_with_user_follower_count,engaged_with_user_following_count,enaging_user_follower_count,enaging_user_following_count,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,language_022EC308651FACB02794A8147AEE1B78,language_0331BF70E606D62D92C96CE9AD71A7CF,language_06BEAB41D66CCFF329D1ED8BA120A6C2,...,enaging_user_id_0023210A085B96DE818A3A5CE8C03BD5,engagee_follows_engager_False,engagee_follows_engager_True,day_0,day_1,day_2,day_3,day_4,day_5,day_6
0,3.112946e-05,0.000313,0.257540,0.006159,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,3.527001e-04,0.017507,0.000160,0.000781,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,1.256527e-03,0.001716,0.000019,0.000288,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,2.693572e-05,0.000053,0.000028,0.000298,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,1.253371e-03,0.001717,0.000028,0.000298,0,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,3.005664e-06,0.000092,0.004300,0.002406,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
9996,7.272467e-04,0.017154,0.000263,0.000679,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
9997,4.222143e-03,0.000782,0.000263,0.000679,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
9998,8.954929e-07,0.000059,0.001023,0.001851,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [26]:
from sklearn.linear_model import LassoCV
import numpy as np
clf = LassoCV().fit(X_numerical, y)
importance = np.abs(clf.coef_)
print(importance)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.         0.         0.         ... 0.00047035 0.00601686 0.00178939]


In [27]:
[(X_numerical.columns[i], e) for i, e in enumerate(importance) if e != 0]

[('tweet_type_Retweet', 0.027983871971784677),
 ('tweet_type_TopLevel', 0.007218718332724599),
 ('language_022EC308651FACB02794A8147AEE1B78', 0.0023360693086570375),
 ('language_06D61DCBBE938971E1EA0C38BD9B5446', 0.002485239042574822),
 ('language_167115458A0DBDFF7E9C0C53A83BAC9B', 0.012303619498243178),
 ('language_22C448FF81263D4BAF2A176145EE9EAD', 0.0077560473682354684),
 ('language_4DC22C3F31C5C43721E6B5815A595ED6', 0.01681502141892007),
 ('language_B9175601E87101A984A50F8A62A1C374', 0.01578856440937213),
 ('language_D3164C7FBCF2565DDF915B1B3AEFB1DC', 0.004657722590492614),
 ('language_ECED8A16BE2A5E8871FD55F4842F16B1', 0.00793368501472443),
 ('language_FA3F382BC409C271E3D6EAF8BE4648DD', 0.0014549414946094057),
 ('enaging_user_id_0006F4CE8EABF29D99DC4A35E1C227A0', 0.08534017405811002),
 ('engagee_follows_engager_False', 0.026944096301003542),
 ('engagee_follows_engager_True', 7.261645688978867e-18),
 ('day_0', 0.0029403627696555253),
 ('day_3', 0.0017408325448172856),
 ('day_4', 0.